In [ ]:
# I import all the packages I need
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

In [ ]:
# I upload the dataset related to the ratings that contains the rating (out of 5) by each user for some movies

ratings= pd.read_csv(ValentinaCoppi/IT-coding-project/ratings.csv)
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
# I upload the dataset that contains the title of the movies and their genre

movies= pd.read_csv(ValentinaCoppi/IT-coding-project/movies.csv)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


My aim is to create a collaborative filtering based on the cosine distance between the users, in order to fill in the gaps of the rating for the unseen movies, and propose the user the best movies he may like.

In [4]:
# I calculate the mean rating for each user
def meanbyuser(user):
    return ratings.groupby('userId')[['rating']].mean().rating[user]

In [5]:
mean_rating= ratings.groupby('userId')[['rating']].mean()

In [6]:
#I change the names at the original dataset and the userId column

x = ratings.copy()
y = ratings['userId']


In [7]:
# I need to transform the training dataset in a matrix that has users has row index and movie as column index, and fill it in with the ratingd

df_ratings = x.pivot(index='userId', columns= 'movieId', values='rating')

In [8]:
df_ratings

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_ratings0= df_ratings.copy()

In [10]:
# I want to normalize all the ratings, subtracting the mean value for each user, in order to make the ratings more fair

for i in range(1,len(mean_rating['rating']+1)):
    df_ratings0.loc[i,:]=np.array(df_ratings0.loc[i,:])-mean_rating['rating'][i] 


In [11]:
df_ratings0

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,NaN,-0.366379,NaN,NaN,-0.366379,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.363636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,-1.157399,NaN,NaN,NaN,NaN,NaN,-1.157399,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,0.213904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,-0.634176,-1.134176,-1.134176,NaN,NaN,NaN,NaN,NaN,NaN,0.865824,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# I want that NaN are filled with 0, or I can't work on the matrix

df_ratings1 = df_ratings0.copy().fillna(0)
df_ratings1.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,-0.366379,0.0,-0.366379,0.0,0.0,-0.366379,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.363636,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# I want to calculate the cosine similarity of the ratings

similarity_matrix = cosine_similarity(df_ratings1, df_ratings1)
similarity_matrix_df = pd.DataFrame(similarity_matrix, index=df_ratings.index, columns=df_ratings.index)

In [14]:
similarity_matrix_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.001265,0.000553,0.048419,0.021847,-0.045497,-0.006200,0.047013,0.019510,-0.008754,...,0.018127,-0.017172,-0.015221,-0.037059,-0.029121,0.012016,0.055261,0.075224,-0.025713,0.019612
2,0.001265,1.000000,0.000000,-0.017164,0.021796,-0.021051,-0.011114,-0.048085,0.000000,0.003012,...,-0.050551,-0.031581,-0.001688,0.000000,0.000000,0.006226,-0.020504,-0.006001,-0.060091,-0.004277
3,0.000553,0.000000,1.000000,-0.011260,-0.031539,0.004800,0.000000,-0.032471,0.000000,0.000000,...,-0.004904,-0.016117,0.017749,0.000000,-0.001431,-0.037289,-0.007789,-0.013001,0.000000,0.011649
4,0.048419,-0.017164,-0.011260,1.000000,-0.029620,0.013956,0.058091,0.002065,-0.005874,0.051590,...,-0.037687,0.063122,0.027640,-0.013782,0.040037,0.020590,0.014628,-0.037569,-0.017884,0.003355
5,0.021847,0.021796,-0.031539,-0.029620,1.000000,0.009111,0.010117,-0.012284,0.000000,-0.033165,...,0.015964,0.012427,0.027076,0.012461,-0.036272,0.026319,0.031896,-0.001751,0.093829,-0.003841


In [38]:
def Reccomand(picked_userid):
    # Remove picked user ID from the candidate list
    similarity_matrix_df.drop(index=picked_userid, inplace=False)
   
    # Number of similar user
    n = 10
    # User similarity threashold
    user_similarity_threshold = 0.09
    # Get top n similar users
    similar_users = similarity_matrix_df[similarity_matrix_df[picked_userid]>user_similarity_threshold][picked_userid].sort_values(ascending=False)[:n]

    # Movies that the target user has watched
    picked_userid_watched = df_ratings0[df_ratings0.index == picked_userid].dropna(axis=1, how='all')
    picked_userid_watched1 = df_ratings[df_ratings.index == picked_userid].dropna(axis=1, how='all')
    # Movies that similar users watched. Remove movies that none of the similar users have watched
    similar_user_movies = df_ratings0[df_ratings0.index.isin(similar_users.index)].dropna(axis=1, how='all')
    # Remove the watched movie from the movie list
    similar_user_movies.drop(picked_userid_watched.columns,axis=1, inplace=True, errors='ignore')
    
    # A dictionary to store item scores
    item_score = {}
    # Loop through items
    for i in similar_user_movies.columns:
      # Get the ratings for movie i
        movie_rating = similar_user_movies[i]
      # Create a variable to store the score
        total = 0
      # Create a variable to store the number of scores
        count = 0
      # Loop through similar users
        for u in similar_users.index:
        # If the movie has rating
            if pd.isna(movie_rating[u]) == False:
          # Score is the sum of user similarity score multiply by the movie rating
              score = similar_users[u] * movie_rating[u]
          # Add the score to the total score for the movie so far
              total += score
          # Add 1 to the count
              count +=1
      # Get the average score for the item
        item_score[i] = total / count+ meanbyuser(picked_userid)
    # Convert dictionary to pandas dataframe
    item_score = pd.DataFrame(item_score.items(), columns=['movieId', 'movie_score'])
    item_score['movie_score'] = item_score['movie_score'].apply(lambda x: float("{:.2f}".format(x)))
    # Sort the movies by score
    ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)
    best_movies= pd.merge( ranked_item_score, movies, on= 'movieId')
    picked_userid_watched1= pd.merge(picked_userid_watched1.T, movies, on= 'movieId')
    picked_userid_watched2=picked_userid_watched1.sort_values(by=[picked_userid], ascending=False)
    picked_userid_watched2.rename(columns = {picked_userid:'Rate'}, inplace = True)
    # Select top m movies
    m = 20
    best_movies.head(m).to_excel("output.xlsx")
    picked_userid_watched2.head(m).to_excel('watched.xlsx')
    return best_movies.head(m)
    return picked_userid_watched2.head(m)

In [37]:
window = tk.Tk()
window.title("Choose UserId")
window.geometry('600x400')
window.configure(bg='#333333')

def login():
    username = range(1,len(df_ratings)+1)
    try:
        if int(username_entry.get()) in username:
            open_secondary_window()
    except:
        messagebox.showinfo(title="Error", message="Invalid UserId.")
        
        
def look():
    username = range(1,len(df_ratings)+1)
    try:
        if int(username_entry.get()) in username:
            open_third_window()
    except:
        messagebox.showinfo(title="Error", message="Invalid UserId.")

frame = tk.Frame(bg='#333333')

    
def open_secondary_window():
    # Create secondary (or popup) window.
    secondary_window = tk.Toplevel()
    secondary_window.title("Reccomanded movies")
    secondary_window.config(bg='#333333')
    
    Reccomand(int(username_entry.get()))
    df= pd.read_excel("output.xlsx", usecols= 'C, D,E')
    # Extract number of rows and columns
    n_rows = df.shape[0]
    n_cols = df.shape[1]

    # Extracting columns from the data and
    # creating text widget with some
    # background color
    column_names = df.columns
    i=0
    text = tk.Text(secondary_window, width=15, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=0)
    text.insert(tk.INSERT, 'Possible rate')

    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=1)
    text.insert(tk.INSERT, 'Title')
    
    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=2)
    text.insert(tk.INSERT, 'Genre')

    # adding all the other rows into the grid
    for i in range(n_rows):

        text = tk.Text(secondary_window, width=15, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=0)
        text.insert(tk.INSERT, df.loc[i][0])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=1)
        text.insert(tk.INSERT, df.loc[i][1])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=2)
        text.insert(tk.INSERT, df.loc[i][2])
      

    secondary_window.grab_set()  # Modal.
    
    
def open_third_window():
    # Create secondary (or popup) window.
    secondary_window = tk.Toplevel()
    secondary_window.title("Reccomanded movies")
    secondary_window.config(bg='#333333')
    
    Reccomand(int(username_entry.get()))
    df= pd.read_excel("watched.xlsx", usecols= 'C, D,E')
    # Extract number of rows and columns
    n_rows = df.shape[0]
    n_cols = df.shape[1]

    # Extracting columns from the data and
    # creating text widget with some
    # background color
    column_names = df.columns
    i=0
    text = tk.Text(secondary_window, width=15, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=0)
    text.insert(tk.INSERT, 'Possible rate')

    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=1)
    text.insert(tk.INSERT, 'Title')
    
    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=2)
    text.insert(tk.INSERT, 'Genre')
    

    # adding all the other rows into the grid
    for i in range(n_rows):

        text = tk.Text(secondary_window, width=15, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=0)
        text.insert(tk.INSERT, df.loc[i][0])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=1)
        text.insert(tk.INSERT, df.loc[i][1])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=2)
        text.insert(tk.INSERT, df.loc[i][2])
    
    
    secondary_window.grab_set()  # Modal.

# Creating widgets
login_label = tk.Label(
    frame, text="  Choose the UserId desired", bg='#333333', fg='#00CCCC', font=("Arial", 30))
username_label = tk.Label(
    frame, text="Username", bg='#333333', fg="#FFFFFF", font=("Arial", 16))
username_entry= tk.Entry(frame, font=("Arial", 16))
login_button = tk.Button(
    frame, text="Reccomand", bg="#00CCCC", fg="#FFFFFF", font=("Arial", 16), command=login)
liked_button = tk.Button(
    frame, text="My movies", bg="#00CCCC", fg="#FFFFFF", font=("Arial", 16), command=look)

# Placing widgets on the screen
login_label.grid(row=0, column=0, columnspan=3, sticky="news", pady=40)
username_label.grid(row=1, column=0)
username_entry.grid(row=1, column=1,columnspan=2, pady=20)
login_button.grid(row=3, column=1, columnspan=1, pady=10)
liked_button.grid(row=4, column=1, columnspan=1, pady=10)

frame.pack()

window.mainloop()